# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import apikey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# File to Load (Remember to Change These)
csv_path = "../WeatherPy/output_data/cities.csv"

# Read Purchasing File and store into Pandas data frame
city_data_df = pd.read_csv(csv_path)
city_data_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=apikey)

In [37]:
#Store Lat and Lng into coords
coords = city_data_df[['Lat','Lng']].astype(float)
humidity = city_data_df['Humidity'].astype(float)

#Create a heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coords, humidity)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
#Create new df
smallerweather_df = pd.DataFrame(city_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness", "Country", "Lat", "Lng"])
len(smallerweather_df)

#Drop null values (there are no null values)
cleanweather_df = smallerweather_df.dropna(axis=0, how='all')
len(cleanweather_df)

#Narrow down df to max temp >80, wind speed <10, and 0 clouidness
smallerweather_df = smallerweather_df[smallerweather_df['Max Temp'] >80]
smallerweather_df = smallerweather_df[smallerweather_df['Wind Speed'] < 10]
smallerweather_df = smallerweather_df[smallerweather_df['Cloudiness'] == 0]
smallerweather_df.head()

,City,Max Temp,Wind Speed,Cloudiness,Country,Lat,Lng
26,colonelganj,94.20,5.08,0,IN,27.13,81.70
125,arraial do cabo,84.20,4.70,0,BR,-22.97,-42.02
142,sarai mir,86.00,3.36,0,IN,26.02,82.92
244,pasni,86.82,9.17,0,PK,25.26,63.48
316,gunjur,80.60,9.17,0,GM,13.18,-16.76


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
#Create hotel_df
hotel_df = pd.DataFrame(smallerweather_df, columns=['City', 'Country', 'Lat', 'Lng'])
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,colonelganj,IN,27.13,81.70,
125,arraial do cabo,BR,-22.97,-42.02,
142,sarai mir,IN,26.02,82.92,
244,pasni,PK,25.26,63.48,
316,gunjur,GM,13.18,-16.76,
416,kirkuk,IQ,35.47,44.40,
518,bonito,BR,-21.13,-56.49,
527,tomatlan,MX,19.93,-105.25,
543,yabrud,SY,33.97,36.66,


In [31]:
# find the closest restaurant of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create for loop to go through each row
for i, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel",
        "radius": 5000,
        "key": apikey
    }
    
    response = requests.get(base_url, params=params)
    places_data = response.json()
    
    print(f"Results for index {i}: {city}.")
    
    try:
        print(places_data["results"][0]['name'])
        hotel_df.loc[i, "Hotel Name"] = places_data["results"][0]['name']
        
    except (KeyError, IndexError):
        print("Results not found.")
        
    print("----------")
print("-------Done--------")

Results for index 26: colonelganj.
Anand Hotel
----------
Results for index 125: arraial do cabo.
Orlanova Boutique Hotel
----------
Results for index 142: sarai mir.
umar hotel
----------
Results for index 244: pasni.
Juddi Hotel
----------
Results for index 316: gunjur.
Nemasu Ecolodge
----------
Results for index 416: kirkuk.
Hotel snubal
----------
Results for index 518: bonito.
Pousada Arte da Natureza
----------
Results for index 527: tomatlan.
Hotel Misión San Miguel
----------
Results for index 543: yabrud.
Results not found.
----------
-------Done--------


In [32]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,colonelganj,IN,27.13,81.70,Anand Hotel
125,arraial do cabo,BR,-22.97,-42.02,Orlanova Boutique Hotel
142,sarai mir,IN,26.02,82.92,umar hotel
244,pasni,PK,25.26,63.48,Juddi Hotel
316,gunjur,GM,13.18,-16.76,Nemasu Ecolodge
416,kirkuk,IQ,35.47,44.40,Hotel snubal
518,bonito,BR,-21.13,-56.49,Pousada Arte da Natureza
527,tomatlan,MX,19.93,-105.25,Hotel Misión San Miguel
543,yabrud,SY,33.97,36.66,


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))